In [2]:
import mediapipe as mp
import os
import cv2
import numpy as np
import math
from IPython.display import clear_output
import time
from pynput.keyboard import Controller, Key

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, result

In [5]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return lh

In [6]:
def extract_thumb_keypoints(results):
    keypoints = []
    for data_point in results.left_hand_landmarks.landmark:
        keypoints.extend([data_point.x, data_point.y, data_point.z])
    return keypoints

In [7]:
def draw_landmarks_right(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4), #Circle 
                                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=1)) #Lines

In [8]:
def draw_landmarks_left(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), #Circle
                                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)) #Lines

In [9]:
class Button():
    def __init__(self, txt, pos, size = 38):
        self.size = size
        self.pos = pos
        self.text = txt

        
        

In [10]:
def place_keys(img, btn, keys, i, j):   
    for key in keys:
        (img, key, [i, j])
        i += 100
    return img

In [11]:
def click_btn(image, btn, il, text, key):
    if abs(il[0]-btn.pos[0]) <= btn.size and abs(il[1]-btn.pos[1]) <= btn.size:
        mask = image.copy()
        cv2.circle(mask, btn.pos, btn.size, (255,255,255), thickness=-1)
        image = cv2.addWeighted(mask, 1, image, 1 - 1, 0)
        if btn.text == "<-":
            if text != "":
                text = text[:-1]
                key.press(Key.backspace)
        elif btn.text == "_":
            text += " " 
            key.press(Key.space)
        else:
            text += btn.text
            key.press(btn.text)
    return image, text

In [12]:

def activate_btn(image, btn, il, ml, text, key):
    if abs(il[0]-btn.pos[0]) <= btn.size and abs(il[1]-btn.pos[1]) <= btn.size:
        mask = image.copy()
        cv2.circle(mask, btn.pos, btn.size, (255,255,255), thickness=-1)
        image = cv2.addWeighted(mask, 0.5, image, 1 - 0.5, 0)
        d = math.dist(il, ml)
        if d < 50:
            
            image, text = click_btn(image, btn, il, text, key)
            time.sleep(0.2)
    return image, text
                    

In [13]:
cap = cv2.VideoCapture(0)

keys = [['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'],
        ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L'],
        ['Z', 'X', 'C', 'V', 'B', 'N', 'M', '_', '<-']]

final_text = ""
keyboard = Controller()
btnList = []

y = 300

for i in range(3):
    x = 100
    for key in keys[i]:
        btnList.append(Button(key, (x, y)))
        x += 100
    y += 100


    

try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        while cap.isOpened():
            ret, frame = cap.read()
            clear_output(wait=True)
            image = cv2.resize(frame,(1080, 784))
            frameW, frameH = 1080, 784
            # image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image, results = mediapipe_detection(image, holistic)
            left_landmarks = results.right_hand_landmarks
            right_landmarks = results.left_hand_landmarks

            for btn in btnList:
                mask = image.copy()
                cv2.circle(mask, btn.pos, btn.size, (0,0,0), thickness=-1)
                image = cv2.addWeighted(mask, 0.8, image, 1 - 0.8, 0)
                cv2.putText(image, btn.text, (btn.pos[0]-17, btn.pos[1]+19), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 2)

            if left_landmarks:
                tlx = int(left_landmarks.landmark[4].x*frameW)
                tly = int(left_landmarks.landmark[4].y*frameH)
                ilx = int(left_landmarks.landmark[8].x*frameW)
                ily = int(left_landmarks.landmark[8].y*frameH)
                cv2.circle(image, (ilx, ily), 8, (255, 0, 255), thickness=2)
                cv2.circle(image, (tlx, tly), 8, (255, 0, 255), thickness=2)

                for btn in btnList:
                    image, text = activate_btn(image, btn, [ilx, ily], [tlx, tly], final_text, keyboard)
                    final_text = text
                    
                    # if right_landmarks:
                    #     irx = int(right_landmarks.landmark[4].x*frameW)
                    #     iry = int(right_landmarks.landmark[4].y*frameH)
                    #     mrx = int(right_landmarks.landmark[8].x*frameW)
                    #     mry = int(right_landmarks.landmark[8].y*frameH)

                    #     d = math.dist([irx, iry], [mrx, mry])
                    #     print(d)
                    #     if d < 40:
                    #         image = click_btn(image, btn.size, i, 300, ilx, ily)
                    #         image = click_btn(image, btn.size, i, 400, ilx, ily)
                    #         image = click_btn(image, btn.size, i, 500, ilx, ily)
            cv2.rectangle(image, (100, 100), (900, 200), (0, 0, 0), cv2.FILLED)
            cv2.putText(image, final_text, (120, 150), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)
                    

            # draw_landmarks_left(image, results)
            # draw_landmarks_right(image, results)
            
            # drawKey(frame,(255,255,255), (0,0,0),0.1, fontScale=0.5)
            cv2.imshow("Hand Tracking", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
finally:
    cap.release()
    cv2.destroyAllWindows()

107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349
107.00467279516349


In [14]:
for i in range(100, 1001, 100):
    print(i)

100
200
300
400
500
600
700
800
900
1000
